# Problem 4 : Estimation - BLP

In [27]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra,StatsFuns

Random.seed!(6789998212);

In [28]:
# load in csv
df = DataFrame(CSV.File("../data/ps1_ex4.csv"));

In [29]:
# simulate individual taste shocks from N(μ,Σ)
draw_sim = function(μ, Σ, N) # return N x L matrix
    # draw shocks
    v = rand(MvNormal(μ, Σ), N)
    
    return v
end

#11 (generic function with 1 method)

In [231]:
# get data
n_markets = 100
n_sim = 50

x_t = []
for t in 1:n_markets 
   push!(x_t, Array(df[df[!,:market].==t, [:p, :x]]))
end

s_t = []
for t in 1:n_markets 
    push!(s_t, Array(df[df[!,:market].==t, [:shares]]))
end

z_t = []
for t in 1:n_markets 
    push!(z_t, Array(df[df[!,:market].==t, [:z1, :z2, :z3, :z4, :z5, :z6, :x]]))
end

x_jt = Array(df[df[!,:market] .<= n_markets,[:p, :x]]);

z_jt = Array(df[df[!,:market] .<= n_markets,[:z1, :z2, :z3, :z4, :z5, :z6, :x]]);

v = draw_sim([0;0], [1 0;0 1], n_sim)

2×50 Matrix{Float64}:
 -1.6012    -0.579413  -0.0137741  …  -0.324713  -0.945904  0.128032
 -0.130687  -0.539617  -1.3303        -1.12882    2.14096   0.215248

# Part 1: BLP

## Inner loop
`get_shares` calculates the shares of each product in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a matrix of size $J \times 2$; and $v$ should be a vector of length $L$.

`delta_contraction` iterates the $\delta_{jt}$ in a particular market $t$. $\delta$ should be a vector of length $J$; $x$ should be a vector of characteristics with length $J$; $s$ should be a vector of observed shares with length $J$; $v$ should be a vector of length $L$. 

`market_iterate` performs the contraction over each $t$ markets, it recoves $\delta_{jt}$, which is a vector of length $J \times T$.

In [7]:
# get shares in a market given some fixed gamma and delta
get_shares = function(δ, Γ, x, v)
    # we want to get share_{jt} using simulated values of $v_i$ (drawn above)
    # shares should be vector of length J
    numerator = exp.(δ .+ x * Γ * v)
    adj = maximum(numerator, dims = 1)
    denominator = sum((numerator ./ adj), dims = 1) .+ (1 ./ adj)
    shares = sum((numerator ./ adj) ./ denominator, dims = 2) ./ size(v)[2]
    
    return shares
end

# inner loop: contraction to find δ
delta_contraction = function(δ₀, Γ, s, x, v, tol = 1e-12, max_iter = nothing)

    # here δ is a vector of length J
    δ = δ₀
    err = 1000
    n = 0
    maxed_iter = false
    
    while (err > tol) && (maxed_iter === false)
        δ_old = δ
        
        # update delta
        δ = δ_old + log.(s) - log.(get_shares(δ_old, Γ, x, v))
        
        # difference 
        err = maximum(abs.(δ - δ_old)) 
        
        # (optional) max iterations block
        n += 1
        if max_iter !== nothing
            maxed_iter = (n == max_iter)
        end
    end
    
    return δ
end

# iterate over each market
market_iterate = function(Γ, s_t, x_t, v, tol = 1e-12, max_iter = nothing)
   
    δ = []
    for t in 1:size(s_t)[1]
        s = s_t[t]
        x = x_t[t]
        δ₀ = ones(size(s)[1])
        push!(δ, delta_contraction(δ₀, Γ, s, x, v, tol, max_iter) ) 
    end
    return δ
end

#7 (generic function with 3 methods)

## Outer loop
`residuals` does IV-GMM using the provided weighting matrix. z_jt should be a matrix of $Z$ excluded and included intruments of size $TJ \times Z$. Returns linear parameters (vector of length $2$) and $\xi_{jt}$ residuals (vector of length $J \times T$)

`gmm_objective` Reads in $TJ$-length vector $x$_jt and $TJ \times Z$ matrix $z$_jt. Calculates sample moments (size of instrument vector, $Z$) and optimal weighting matrix ($Z \times Z$). Returns scalar objective and matrix.

## Gradient
Estimator is $$\nabla G(\theta) = 2(Z'J_\theta)'W(Z'\xi(\theta))$$
-$Z$ is $JT \times Z$ matrix if instruments

-$\xi$ is $JT \times 1$ matrix of unobserved mean utilities

-$W$ is $Z \times Z$ 

$$ W = \left[(\xi(\theta) \circ Z)' (\xi(\theta) \circ Z) \right]^{-1}$$

-$J_\theta$ is $JT \times 3$

$$ J_\theta = -f_\xi^{-1} f_\theta$$

- For each t, $f_\xi$ is a $J \times J$ matrix: $\left\{\frac{\partial s_{ij}}{\partial \xi_k}\right\}_{j,k}$

$$ \frac{\partial s_{ij}}{\partial \xi_k} = -s_{ij}s_{ik}, \quad \frac{\partial s_{ij}}{\partial \xi_j} = s_{ij}(1-s_{ij}) $$

- For each t, $f_\theta$ is a $J \times 3$ matrix: 
$$\begin{bmatrix} s_{ij}s_{i0}\left(p_j - \sum_k s_{ik}p_k \right)\nu_{1i}  &  s_{ij}s_{i0}\left(x_j - \sum_k s_{ik}x_k \right)\nu_{1i} & s_{ij}s_{i0}\left(x_j - \sum_k s_{ik}x_k \right)\nu_{2i} \end{bmatrix}_{j} $$

All matrices are stacked $J \times \cdot$ over $T$ markets

In [229]:
# returns residuals for a given δ, estimates linear parameters given instruments
resid = function(δ_jt, x_jt, z_jt, W)
    # iv-gmm
    θ₁ = inv(x_jt' * z_jt * W * z_jt' * x_jt) * (x_jt' * z_jt * W * z_jt' * δ_jt)
    ξ_jt = δ_jt - x_jt * θ₁
    
    return ξ_jt, θ₁ 
    
end

# calculates gmm objective for outer loop
function gmm_objective(ξ_jt, z_jt, W)   
    # empirical moments, weighting matrix
    g = (ξ_jt' * z_jt) / size(ξ_jt)[1] 
    
    # gmm objective
    G = g * W * g'
    
    return G
end

# performs outer loop
function outer_loop(θ₂, s_t, x_t, x_jt, z_jt, v, W, tol = 1e-12, max_iter = nothing)
    # Pass through guess
    Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]] # lower triangular
    println(Γ)
    
    # Perform inner loop
    δ = market_iterate(Γ, s_t, x_t, v, tol, max_iter)
    
    # convert to JT x 1 (stacked J x 1 vectors for each t)
    δ_jt = vec(reduce(hcat,δ)) 
    
    # intermediate step
    ξ_jt, θ₁ = resid(δ_jt, x_jt, z_jt, W)
    
    # gmm step
    G = gmm_objective(ξ_jt, z_jt, W)
    
    println(G)
    
    return G
end

# Steps to calculate gradient...
# have data s_t, x_t, z_t
# first step will return θ           -> Γ  x
# 1. run market_iterate() with Γ     -> δ  x
# 2. run resid() with δ              -> ξ x
# 3. calculate W with ξ and Z        -> W x
# 4. calculate J with δ and Γ*       -> J  x
# *(for each i, calculate s_ij vector, do elementwise mult with p_j, v, and sum to get f_xi loop through j,k for f_theta)
# 5. run gradient() with J, W, ξ, Z  -> ∇

# helper function in gradient call: for each market get Jacobian of ξ(θ)
function jacobian_xi(δ, Γ, x, v)
    # need individual shares
    numerator = exp.(δ .+ x * Γ * v)
    adj = maximum(numerator, dims = 1)
    denominator = sum((numerator ./ adj), dims = 1) .+ (1 ./ adj)
    shares = (numerator ./ adj) ./ denominator # J x L
    
    # calculate partials of f(θ) = s - S(ξ,θ), denoted fξ and fθ
    fξ_store = []
    fθ_store = [] 
    for i = 1:size(v)[2]
        s_i = shares[:,i]
        s_i0 = 1 - sum(s_i)
        v_i = v[:,i]
        
        fξ_i = - s_i * s_i' + diagm(s_i)
        
        fθ₁ = s_i .* (x[:,1] .- (s_i' * x[:,1])) .* v[1]
        fθ₂ = s_i .* (x[:,2] .- (s_i' * x[:,2])) .* v[1]
        fθ₃ = s_i .* (x[:,2] .- (s_i' * x[:,2])) .* v[2]
        fθ_i = hcat(fθ₁, fθ₂, fθ₃)
        
        push!(fξ_store, fξ_i)
        push!(fθ_store, fθ_i)        
    end
    
    # calculate Jacobian
    J = -1 .* inv(mean(fξ_store)) * mean(fθ_store)
   
    return J
end
    
function gmm_gradient!(θ₂, s_t, x_t, x_jt, z_jt, v, W, ∇, tol = 1e-12, max_iter = nothing)
    # Pass through guess
    Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]] # lower triangular
    
    println(1, Γ)
    
    # Recover model objects from estimates parameters: Γ, and data: s_t, x_t, z_t, and v (simulated)    
    # δ(θ)
    δ = market_iterate(Γ, s_t, x_t, v, tol, max_iter)
    
    println(2,δ)
    
    # ξ(θ)
    δ_jt = vec(reduce(hcat,δ)) 
    ξ_jt = resid(δ_jt, x_jt, z_jt, W)[1]
    ξ_t = reshape(ξ_jt, 6, Int64(size(ξ_jt)[1] / 6))
    
    println(3, ξ_t)
    
    # Analytic matrices
    # Jacobian
    J_t = []
    for t = 1:size(x_t)[1]
        push!(J_t, jacobian_xi(δ[t], Γ, x_t[t], v))
    end
    # J = reduce(vcat, J_t) # flatten to JT x 3 matrix
    
    println(4, mean(J_t))
    
    # Weighting (note: put outside, we want to fix W through run)
    # W = inv((z_jt .* ξ_jt)' * (z_jt .* ξ_jt)) * size(ξ_jt)[1]
    
    # Calculate gradient
    ∇_t = []
    for t in 1:size(s_t)[1]
        push!(∇_t,  2 .* (z_t[t]' * J_t[t])' * W * (z_t[t]' * ξ_t[:,t]))
    end
    ∇ .= mean(∇_t)
    
    print(5,∇)

    return ∇
end

gmm_gradient! (generic function with 3 methods)

In [237]:
δ₀ = ones(6)
params0 = ones(3)
Γ = [params0[1] 0 ; params0[2] params0[3]]

w = inv(z_jt' * z_jt);

tol = 1e-14;
max_iter = nothing;

f(θ₂) = outer_loop(θ₂, s_t, x_t, x_jt, z_jt, v, w, tol, max_iter)
g!(G,θ₂) = gmm_gradient!(θ₂, s_t, x_t, x_jt, z_jt, v, w, G, tol, max_iter)

g! (generic function with 1 method)

In [238]:
# @time o = Optim.optimize(f, g!,params0, LBFGS())
@time o = Optim.optimize(f, params0, LBFGS(), Optim.Options(show_trace = true, show_every = 100))

[1.0000060554544523 0.0; 1.0 1.0]
1.171806166128546
[0.9999939445455476 0.0; 1.0 1.0]
1.171806260105454
[1.0 0.0; 1.0000060554544523 1.0]
1.171806723117817
[1.0 0.0; 0.9999939445455476 1.0]
1.1718057031178335
[1.0 0.0; 1.0 1.0000060554544523]
1.1718074974883133
[1.0 0.0; 1.0 0.9999939445455476]
1.1718049287484784
[1.0 0.0; 1.0 1.0]
1.1718062131170648
Iter     Function value   Gradient norm 
     0     1.171806e+00     2.121013e-01
 * time: 0.0
[1.0077657932119544 0.0; 0.9157784117108019 0.787898674241917]
1.1213897469260083
[1.0077535883261417 0.0; 0.9157784117108019 0.787898674241917]
1.1213899149007132
[1.007759690769048 0.0; 0.9157844671652543 0.787898674241917]
1.1213903412117172
[1.007759690769048 0.0; 0.9157723562563495 0.787898674241917]
1.1213893206165004
[1.007759690769048 0.0; 0.9157784117108019 0.7879047296963694]
1.1213910057972747
[1.007759690769048 0.0; 0.9157784117108019 0.7878926187874645]
1.1213886560337778
[1.007759690769048 0.0; 0.9157784117108019 0.787898674241917]


[1.9222959272375548 0.0; -0.03425396892602081 0.06065649088938514]
0.9621829640630329
[1.9222726466276665 0.0; -0.03425396892602081 0.06065649088938514]
0.9621828951596758
[1.9222842869326107 0.0; -0.03424791347156842 0.06065649088938514]
0.9621830585661293
[1.9222842869326107 0.0; -0.0342600243804732 0.06065649088938514]
0.9621828006612102
[1.9222842869326107 0.0; -0.03425396892602081 0.06066254634383753]
0.9621830881009881
[1.9222842869326107 0.0; -0.03425396892602081 0.06065043543493275]
0.9621827711351367
[1.9222842869326107 0.0; -0.03425396892602081 0.06065649088938514]
0.9621829296105215
[1.695060278791018 0.0; -0.14605018123510788 -0.005521587972383571]
0.9596194230567162
[1.6950397501947037 0.0; -0.14605018123510788 -0.005521587972383571]
0.9596194248822196
[1.6950500144928609 0.0; -0.14604412578065548 -0.005521587972383571]
0.9596194245214893
[1.6950500144928609 0.0; -0.14605623668956028 -0.005521587972383571]
0.9596194234240871
[1.6950500144928609 0.0; -0.14605018123510788 -0

 * Status: success

 * Candidate solution
    Final objective value:     9.596186e-01

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 3.11e-06 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.83e-06 ≰ 0.0e+00
    |f(x) - f(x')|         = 1.22e-13 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 1.28e-13 ≰ 0.0e+00
    |g(x)|                 = 2.48e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   101  (vs limit Inf)
    Iterations:    9
    f(x) calls:    29
    ∇f(x) calls:   29


In [235]:
using ForwardDiff
ForwardDiff.gradient(f,θ₂)

ForwardDiff.Dual{ForwardDiff.Tag{typeof(f), Float64}, Float64, 3}[Dual{ForwardDiff.Tag{typeof(f), Float64}}(7.844755012180929e-11,1.0,0.0,0.0) Dual{ForwardDiff.Tag{typeof(f), Float64}}(0.0,0.0,0.0,0.0); Dual{ForwardDiff.Tag{typeof(f), Float64}}(6.183710584737632e-11,0.0,1.0,0.0) Dual{ForwardDiff.Tag{typeof(f), Float64}}(4.7174964023337557e-11,0.0,0.0,1.0)]
Dual{ForwardDiff.Tag{typeof(f), Float64}}(1.1732172907014211,9.621365010647939e-11,7.774407717304635e-11,4.837797888460875e-11)


3-element Vector{Float64}:
 9.621365010647939e-11
 7.774407717304635e-11
 4.837797888460875e-11

In [236]:
∇ = ones(3)
gmm_gradient!(θ₂, s_t, x_t, x_jt, z_jt, v, w, ∇, tol, max_iter)

1[7.844755012180929e-11 0.0; 6.183710584737632e-11 4.7174964023337557e-11]


2Any[[-3.5613903759475187; -0.5139860918637245; -3.6567005557461796; -2.3043077463185; 0.048981387139833577; -2.7140925154853144;;], [-3.9353388107365657; -2.601688183108828; -0.9117918490197798; -4.681129724764062; -2.803109911231315; -0.06226988728476712;;], [-2.5987135370656067; -3.692871475379161; -3.891322414108207; -4.8323057585613824; -0.8387027660851711; -2.3015895727192346;;], [-0.44849882982521083; -1.2502883322278264; -3.954884298467949; -1.1065725296294158; -2.5685899373132663; -0.04908753239963026;;], [-0.8417785287611967; -2.0262883427691074; -0.7708028024285758; -4.704261088621899; -0.9289491286869487; -2.450049579889764;;], [-1.8019891290307393; -1.7557300056461806; -4.587574424087779; -0.22107468692004173; -2.395531205465876; -2.0702531155856048;;], [-1.3562420160645283; -0.651795280246467; -2.949072192093273; -3.2406835426850735; -4.61913472629258; 0.18360475989726777;;], [-0.5375441357134514; -1.4125635358040967; -4.726075912967203; -3.3305648967450034; -5.8246882016

3[-3.5456022884385074 -3.9703574959479533 -2.592351606341894 -0.36506798804075447 -0.8610638420918184 -1.8009523412168218 -1.2786626116387685 -0.44036989216782685 0.3921468514652197 -2.7103848994025634 -1.04892989114282 -1.4583386161577419 -0.22195989066565633 -0.4152913141589497 -4.057244828578532 -2.604031873163498 -1.6928051374289774 -3.6319051922520025 -1.581766812282933 -1.8236987570314447 -1.800738848114037 -1.4544012878157604 -0.8495420189911194 0.45636679097719707 -2.5816577665471088 -0.621647253372765 -0.7254562034147368 -2.1408236777374112 -2.834130964388527 -0.9730089903778352 -4.126042276963975 -2.9158161058586898 -1.6133729571645945 -0.9549984797093632 -2.8953859953985734 -2.6635764120026586 -2.0935681629471126 -2.2793870365211872 -3.3865339332096536 0.022722343225326926 -1.6737555262822956 -3.137215818802428 -1.9804170823951763 -1.4403355032833849 -1.7835981268643097 -1.5036017517157765 -1.591205421957673 0.08816574839433665 -3.05955023159251 -1.416403383161317 -1.8414191

3-element Vector{Float64}:
 86.04552551804397
 38.765997947009694
  3.164000656128222

In [239]:
w

7×7 Matrix{Float64}:
  0.705264   -0.306556    -0.308039   …  -0.0536317    0.0360164
 -0.306556    0.762456    -0.310039      -0.00347183  -0.0148966
 -0.308039   -0.310039     0.735207       0.0331281   -0.0364743
 -0.109018   -0.151796    -0.0847107      0.00258208   0.0431707
 -0.0608856  -0.111811    -0.130583       0.00288287  -0.00640766
 -0.0536317  -0.00347183   0.0331281  …   1.10095      0.0219812
  0.0360164  -0.0148966   -0.0364743      0.0219812    0.491336

In [234]:
θ₂ = o.minimizer
Γ = [θ₂[1] 0 ; θ₂[2] θ₂[3]]

δ = market_iterate(Γ, s_t, x_t, v)
    
δ_jt = vec(reduce(hcat,δ)) 
ξ_jt, θ₁ = resid(δ_jt, x_jt, z_jt, w)
    
S_t = []
for t in 1:size(x_t)[1]
    push!(S_t, get_shares(δ[t], Γ, x_t[t], v))
end
S = mean(reduce(hcat,S_t), dims = 2)

s = mean(reduce(hcat,s_t), dims = 2)

print(θ₁, θ₂)
maximum(reduce(hcat,S_t) - reduce(hcat,s_t))


reshape(ξ_jt,6,Int64(size(ξ_jt)[1] / 6))

[-0.17831303888146594, 0.07459983613889894][7.844755012180929e-11, 6.183710584737632e-11, 4.7174964023337557e-11]

6×100 Matrix{Float64}:
 -3.5456    -3.97036  -2.59235    …  -3.45855   -2.10252   -2.83065
 -0.39771   -2.59021  -3.72191       -0.611346   0.041116  -1.6978
 -3.41571   -0.61966  -3.79179       -0.908962  -3.55455   -5.89333
 -2.18495   -4.15838  -4.60276       -3.89093   -0.605271  -6.01831
  0.762296  -2.5222    0.0925242     -0.836045  -0.903144  -2.39084
 -2.12802    0.166    -2.00955    …  -0.468354  -0.596811   1.25885